In [3]:
import joblib 
import os
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from experiments_helpers import ExperimentResults

if 'src' in os.getcwd():
    os.chdir('..')
    print(os.getcwd())

folder = 'results2'
name = 'torch-fico-gs'

idx = 0
es_base: ExperimentResults = ExperimentResults.load_results_from_file(f'{folder}/{name}_{idx}.joblib')

es_base.pretty_print_robust_vs_base()

ExperimentResults with 18 metrics and 3 artifacts.
############################## Metrics ##############################
------------------------- Base metrics -------------------------
validity: 1.00 (std: 0.00)
proximityL1: 0.29 (std: 0.15)
lof: -1.19 (std: 0.15)
cf_counterfactual_stability: -0.00 (std: 0.12)
generation_time: 0.01 (std: 0.01)
------------------------- Base metrics 2 -------------------------
validity_2: 0.25 (std: 0.43)
proximityL1_2: 0.29 (std: 0.15)
lof_2: -1.19 (std: 0.15)
cf_counterfactual_stability_2: 0.49 (std: 0.19)
------------------------- Robust metrics -------------------------
robust_validity: 1.00 (std: 0.00)
robust_proximityL1: 3.06 (std: 2.23)
robust_lof: -1.70 (std: 0.44)
robust_cf_counterfactual_stability: 0.29 (std: 0.19)
robust_generation_time: 22.86 (std: 15.82)
------------------------- Robust metrics 2 -------------------------
robust_validity_2: 0.75 (std: 0.43)
robust_proximityL1_2: 3.06 (std: 2.23)
robust_lof_2: -1.70 (std: 0.44)
robust_cf_co